In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling n

In [ ]:
import os
import cv2
from google.colab.patches import cv2_imshow
from ultralytics import YOLO

# Mount Google Drive (only for Colab)
from google.colab import drive
drive.mount('/content/drive')

# Load trained YOLOv8 model
MODEL_PATH = "/content/drive/MyDrive/Arecanut_Detection_with_ML/Arecanut_Detection/YOLO_Detection/YOLO_training_results/weights/best.pt"
model = YOLO(MODEL_PATH)

# Output folders
DEFAULT_OUTPUT_FOLDER = "/content/drive/MyDrive/Arecanut_Detection_with_ML/Arecanut_Detection/YOLO_Detection/Results"
OUTPUT_IMAGE_FOLDER = os.path.join(DEFAULT_OUTPUT_FOLDER, "images")
OUTPUT_LABEL_FOLDER = os.path.join(DEFAULT_OUTPUT_FOLDER, "labels")

os.makedirs(OUTPUT_IMAGE_FOLDER, exist_ok=True)
os.makedirs(OUTPUT_LABEL_FOLDER, exist_ok=True)

def detect_arecanut(image_path, filename):
    image = cv2.imread(image_path)
    height, width = image.shape[:2]
    results = model(image)

    found = False
    yolo_labels = []

    for result in results:
        for box in result.boxes:
            class_id = int(box.cls[0])
            conf = float(box.conf[0]) * 100
            label = result.names[class_id]

            if label.lower() == 'arecanut':
                found = True
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                print(f"[INFO] Arecanut detected with {conf:.2f}% confidence at ({x1}, {y1}), ({x2}, {y2})")

                # Draw rectangle
                cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(image, f"{label} {conf:.2f}%", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

                # Convert to YOLO format
                x_center = ((x1 + x2) / 2) / width
                y_center = ((y1 + y2) / 2) / height
                bbox_width = (x2 - x1) / width
                bbox_height = (y2 - y1) / height

                # Round to 6 decimal places
                yolo_labels.append(f"{class_id} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}")

    if found:
        # Save detected image
        output_image_path = os.path.join(OUTPUT_IMAGE_FOLDER, f"detected_{filename}")
        cv2.imwrite(output_image_path, image)
        cv2_imshow(image)

        # Save YOLO label text file
        label_filename = os.path.splitext(filename)[0] + ".txt"
        output_label_path = os.path.join(OUTPUT_LABEL_FOLDER, label_filename)
        with open(output_label_path, "w") as f:
            for line in yolo_labels:
                f.write(line + "\n")
    else:
        print("[INFO] No arecanut detected in the image.")

def detect_arecanut_entry(path):
    supported_extensions = ('.jpg', '.jpeg', '.png', '.bmp')

    if os.path.isdir(path):
        for filename in os.listdir(path):
            full_path = os.path.join(path, filename)
            if filename.lower().endswith(supported_extensions):
                print(f"\n[PROCESSING FOLDER IMAGE] {filename}")
                detect_arecanut(full_path, filename)
            else:
                print(f"[SKIPPED] Not an image file: {filename}")

    elif os.path.isfile(path) and path.lower().endswith(supported_extensions):
        filename = os.path.basename(path)
        print(f"\n[PROCESSING SINGLE IMAGE] {filename}")
        detect_arecanut(path, filename)

    else:
        print("[ERROR] Invalid input. Please provide a valid image or folder path.")

# Example usage:
input_path = "/content/drive/MyDrive/Arecanut_Detection_with_ML/Arecanut_Detection/YOLO_Detection/Testing"  # Change to your test folder or image
detect_arecanut_entry(input_path)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
input_path = "/content/drive/MyDrive/Arecanut_Detection_with_ML/Arecanut_Detection/YOLO_Detection/images/image-21.jpg"  # Image
detect_arecanut_entry(input_path)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
input_path = "/content/drive/MyDrive/Arecanut_Detection_with_ML/Arecanut_Detection/YOLO_Detection/images/image-123.jpg"  # Image
detect_arecanut_entry(input_path)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
input_path=("/content/drive/MyDrive/Arecanut_Detection_with_ML/Arecanut_Detection/YOLO_Detection/Testing/image-1.txt")
detect_arecanut_entry(input_path)

[ERROR] Invalid input. Please provide a valid image or folder path.
